# Extracción de datos a través de *Beautiful Soap 4*

## 1.- Obtención de datos general de los datos.

En un primer paso vamos a hacer llamadas a través de la librería de BeautifulSoap4 para obtener datos generales desde la primera pagina.

Como siempre lo primero es importar las librerías 

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()
folder

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online_webscrapping'

### Conectamos la pagina web para acceder a ella y *arañar* los datos de su página web.

In [3]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
response = requests.get(url)

In [4]:
html = response.content
soup = bs(html, "lxml")

#### a.- Vamos a obtener información de los datos existente en la URL.

In [5]:
all_h3 = soup.find_all("h3")
all_h3

[<h3 class="t2sDiv-titulo hidden text-left color-corporativo">Top ventas en amantis</h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/desliz-lubricante-intimo-agua-100ml/">
 <span>Desliz! Lubricante íntimo de agua 100ml</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/tobogane-hot-rabbit-el-superventas-amantis-mejorado/">
 <span>TOBOGANE HOT RABBIT, el superventas de amantis ¡mejorado!</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/ballenato-tu-vibrador-distancia-aleta-movil-sumergible/">
 <span>BALLENATO, tu vibrador a distancia con aleta móvil y sumergible...</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/tandem-2-flex-vibrador-doble-flexible-mando/">
 <span>TANDEM 2 flex, vibrador doble f

#### b.- Extraemos los nombres desde esta URL.

In [6]:
titulos=soup.find_all("h3")
name=[]
for titulo in titulos[1:]:
    nombre=titulo.get_text(strip=True).split(',')[0]
    name.append(nombre)
print(len(name))
print(name)

24
['Desliz! Lubricante íntimo de agua 100ml', 'TOBOGANE HOT RABBIT', 'BALLENATO', 'TANDEM 2 flex', 'FOXY - Succionador con mango vibrador y orejitas', 'FRESH GIRL', 'MENEO sube y baja', 'TOBOGANE', 'REGGIA', 'LIZO 2', 'PROSTARUDO', 'PULSO AIR - Vibrador palpitador con succionador', 'FOXTAIL', 'TROMPI', 'MASABOOM - El gran masajeador sexual', 'BISOU', 'TOK2 bala vibradora con mando', 'SAZZIA', 'TYPHOON', 'CRISTALINO XL', 'IDYLLIC BOY', 'AVENTURE -  Vibrador con imán y mando a distancia', 'TRIS-TRAS', 'AMADOR - FUCKING MACHINE']


#### c.- Con este codigo extraigo la descripción del producto.

In [7]:
titulos=soup.find_all("h3")
desc=[]

for titulo in titulos[1:]:
    description=titulo.get_text(strip=True).split(', ')[1:]
    if description==[]:
        description=["No hay datos"]
    desc.append(description)
print(len(desc))
print(desc)

24
[['No hay datos'], ['el superventas de amantis ¡mejorado!'], ['tu vibrador a distancia con aleta móvil y sumergible...'], ['vibrador doble flexible con mando'], ['No hay datos'], ['6 Kilos y 40cm de piel real disfrutable'], ['placer realista con control remoto'], ['el vibrador doble más vendido'], ['masturbador masculino doble'], ['Dildo de suave silicona en 3 tamaños'], ['estimulador prostático con cabeza rotadora'], ['No hay datos'], ['plug anal cola de zorro de 35cm'], ['vibrador sumergible ideal para Punto-G'], ['No hay datos'], ['besos por ondas de succión con la mejor vibración'], ['más potente y recargable'], ['masturbador hiperrealista'], ['masturbador masculino giroscópico'], ['gran dildo transparente de 22cm'], ['Plug anal a distancia más intenso'], ['No hay datos'], ['anilla con vibrador para doble penetración TRINITY...'], ['No hay datos']]


Vemos que en algunos casos no hay información. 

En otros casos la separación de la descripción, no se ha separado.

Por este motivo, haremos la extracción unicamente de todo y posteriormente lo trataremos con la librería de *pandas*.

#### d.- Con esto extraigo los links de los productos.

In [8]:
productos = soup.find_all(class_='caption')
lista_URLs = []
for producto in productos[8:]:
    URL_producto = producto.find('a')['href']
    lista_URLs.append(URL_producto)

len(lista_URLs)

25

In [9]:
lista_URLs.pop(0)
len(lista_URLs)

24

In [10]:
lista_URLs[0]

'https://www.amantis.net/desliz-lubricante-intimo-agua-100ml/'

Esta información es importante ya que nos permitirá obtener las direcciones de cada producto para hacer posteriormente la información desde cada producto.

#### e.- Obtenemos la información de precio rebajado de cada producto.

In [11]:
all_price=soup.find_all("span",class_="productSpecialPrice")
price=[]

for precio in all_price:
    item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
    price.append(item_price)
len(price)


24

#### Este es el codigo completo para extraer la información de cada página.

In [12]:
titulos=soup.find_all("h3")
name=[]
desc=[]
price=[]
lista_URLs = []

for titulo in titulos:
    nombre=titulo.get_text(strip=True).split(',')[0]
    name.append(nombre)
    description=titulo.get_text(strip=True).split(', ')[1:]
    if description==[]:
        description=["No hay datos"]
    desc.append(description)

all_price=soup.find_all("span",class_="productSpecialPrice")

for precio in all_price:
    item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
    price.append(item_price)

productos = soup.find_all(class_='caption')
for producto in productos[8:]:
    URL_producto = producto.find('a')['href']
    lista_URLs.append(URL_producto)


name.pop(0)
desc.pop(0)
lista_URLs.pop(0)

df_productos = pd.DataFrame({"Name": name,"Description": desc,"Price":price,"link":lista_URLs})
df_productos

,Name,Description,Price,link
0,Desliz! Lubricante íntimo de agua 100ml,[No hay datos],7.99,https://www.amantis.net/desliz-lubricante-inti...
1,TOBOGANE HOT RABBIT,[el superventas de amantis ¡mejorado!],39.99,https://www.amantis.net/tobogane-hot-rabbit-el...
2,BALLENATO,[tu vibrador a distancia con aleta móvil y sum...,49.99,https://www.amantis.net/ballenato-tu-vibrador-...
3,TANDEM 2 flex,[vibrador doble flexible con mando],49.99,https://www.amantis.net/tandem-2-flex-vibrador...
4,FOXY - Succionador con mango vibrador y orejitas,[No hay datos],39.99,https://www.amantis.net/foxy-vibrador-succiona...
5,FRESH GIRL,[6 Kilos y 40cm de piel real disfrutable],99.99,https://www.amantis.net/fresh-girl-6-kilos-40c...
6,MENEO sube y baja,[placer realista con control remoto],49.99,https://www.amantis.net/meneo-sube-baja-realis...
7,TOBOGANE,[el vibrador doble más vendido],36.99,https://www.amantis.net/tobogane-el-vibrador-d...
8,REGGIA,[masturbador masculino doble],29.99,https://www.amantis.net/reggia-masturbador-mas...
9,LIZO 2,[Dildo de suave silicona en 3 tamaños],17.99,https://www.amantis.net/lizo-2-dildo-suave-sil...


## Vamos a sacar la información de todas las páginas posibles.

Para esto vamos a obtener las URLs, como en el punto d. 

Para extraer los links de los productos y ver si podemos sacar la información de su página concreta.

In [13]:
lista_URLs[0]

'https://www.amantis.net/desliz-lubricante-intimo-agua-100ml/'

In [14]:
# Hacemos un nuevo request para el primer libro: 
r = requests.get(lista_URLs[0])

# Creamos una sopa específica con la info de cada libro
soup_producto = bs(r.text, "lxml")

In [15]:
soup_producto

<!DOCTYPE html>
<html dir="LTR" lang="es">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<title>Desliz! Lubricante íntimo</title>
<base href="https://www.amantis.net/tienda/"/>
<script defer="" src="https://use.fontawesome.com/releases/v5.0.6/js/all.js"></script>
<!--<link href="custom.css" rel="stylesheet">-->
<style type="text/css">/* DO NOT CHANGE THIS FILE -------------------------------------------------- */
/* TO OVER-RIDE ANY ASPECT OF THIS CSS, AMEND THE user.css FILE ------------- */
.piGal {width: 250px !important;}.piGal img {max-width: 250px;height: auto;}li.newItemInCart, li.newItemInCart A {color: #a94442;}.form-control {margin-bottom: 5px;display: inline;}.page-header h1 del {font-size: 0.5em;}.productHolder {border: 1px solid #eee;padding: 10px;border-radius: 4px;-webkit-transition: border .5s ease-in-out;-o-transition: border .5s ease-in-out;t

In [16]:
name = soup_producto.find('h1').text
print(name)

Desliz! Lubricante íntimo de agua 100ml


#### a.- Vamos a entrar en una página general para extraer información de ella.

Principalmente vamos a ver como podemos, con un bucle, sacar todos los datos de cada producto.

In [17]:
page=3
url = "https://www.amantis.net/productos-amantis/"              # lista productos
URL = url+'page' + str(page)+'/'
response = requests.get(URL)
titulos=soup.find_all("h3")
titulos

[<h3 class="t2sDiv-titulo hidden text-left color-corporativo">Top ventas en amantis</h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/desliz-lubricante-intimo-agua-100ml/">
 <span>Desliz! Lubricante íntimo de agua 100ml</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/tobogane-hot-rabbit-el-superventas-amantis-mejorado/">
 <span>TOBOGANE HOT RABBIT, el superventas de amantis ¡mejorado!</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/ballenato-tu-vibrador-distancia-aleta-movil-sumergible/">
 <span>BALLENATO, tu vibrador a distancia con aleta móvil y sumergible...</span>
 </a>
 </h3>,
 <h3 class="h3 group inner list-group-item-heading tdd_listado_nombre">
 <a href="https://www.amantis.net/tandem-2-flex-vibrador-doble-flexible-mando/">
 <span>TANDEM 2 flex, vibrador doble f

Este es el bucle para obtener información anterior de las 5 primeras paginas.

In [18]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
response = requests.get(url)


pages= np.arange(2,5)                                   # La primera pagina tiene una serie de datos que no debemos de recoger.
name=[]
desc=[]
price=[]
lista_URLs = []

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)
            print(nombre)

        all_price=soup.find_all("span",class_="productSpecialPrice")

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)


        productos = soup.find_all(class_='caption')

        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
            print(URL_producto)
            
            
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)
            # print(nombre)

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)

        productos = soup.find_all(class_='caption')

        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
            # print(URL_producto)

print("Nombres: ",len(name))
print("Descripción: ",len(desc))
print("URL: ",len(lista_URLs))
print("Precios: ",len(price))

Pagina 2
Pagina 3
Pagina 4
Nombres:  72
Descripción:  72
URL:  72
Precios:  72


Dado que vemos que hay incongruencias en la longitud del tamaño de cada lista, vamos a realizar un Dataframe unicamente de los datos con la misma longitud.

Esta problemática la solucionaremos más adelante.

In [19]:
df_productos = pd.DataFrame({"Name": name,"Description": desc,"link":lista_URLs})
df_productos.head()

,Name,Description,link
0,Kit de 3 Plugs anales con diamante BLACK STAR ...,[No hay datos],https://www.amantis.net/kit-3-plugs-anales-dia...
1,PRO ANAL - vibrador anal progresivo,[No hay datos],https://www.amantis.net/pro-anal-2-vibrador-anal/
2,AVENTURE - Vibrador con imán y mando a distancia,[No hay datos],https://www.amantis.net/aventure-vibrador-iman...
3,TRIS-TRAS,[anilla con vibrador para doble penetración TR...,https://www.amantis.net/tris-anilla-vibrador-d...
4,Vibrador Líquido con sabor Desliz! VIBRAGEL 30ml,[hormigueo...],https://www.amantis.net/desliz-vibragel-liquid...


#### b.- Verifico que he conseguido las URLs de cada producto, esto es importante para obtener la información de cada producto.

Extraer la información de producto, descripción, enlace y precio tomando los datos desde las URLs de cada producto.

Queda pendiente extraer información de los ratings y los comentarios para establecer un estudio

In [20]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
response = requests.get(url)


pages= np.arange(1, 25)
name=[]
desc=[]
price=[]
lista_URLs = []

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos[1:]:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)

        all_price=soup.find_all("span",class_="productSpecialPrice")

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)


        productos = soup.find_all(class_='caption')

        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
            
            
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        titulos=soup.find_all("h3")

        for titulo in titulos:
            nombre=titulo.get_text(strip=True).split(',')[0]
            name.append(nombre)
            description=titulo.get_text(strip=True).split(', ')[1:]
            if description==[]:
                description=["No hay datos"]
            desc.append(description)

        for precio in all_price:
            item_price=precio.get_text(strip=True).replace(",", ".").split('€')[0]
            price.append(item_price)

        productos = soup.find_all(class_='caption')

        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)




# df_productos = pd.DataFrame({"Name": name,"Description": desc,"Price":price,"link":lista_URLs})
# df_productos.head()


Pagina 1
Pagina 2
Pagina 3
Pagina 4
Pagina 5
Pagina 6
Pagina 7
Pagina 8
Pagina 9
Pagina 10
Pagina 11
Pagina 12
Pagina 13
Pagina 14
Pagina 15
Pagina 16
Pagina 17
Pagina 18
Pagina 19
Pagina 20
Pagina 21
Pagina 22
Pagina 23
Pagina 24


Verificamos que cantidad de datos tenemos al recorrer todos las paginas con productos.

In [21]:
print("Nombres:" ,len(name))
print("descrip:" ,len(desc))
print("precio:" ,len(price))                    #  Se ve que hay un desajuste en el precio al extraer la información de 1 pagina
print("URL:" ,len(lista_URLs))

Nombres: 576
descrip: 576
precio: 576
URL: 576


In [22]:
print("Nombres:\n" ,name[:5])
print("descrip:\n" ,desc[:5])
print("precio:\n" ,price[:5])
print("URL:\n" ,lista_URLs[:3])

Nombres:
 ['Desliz! Lubricante íntimo de agua 100ml', 'TOBOGANE HOT RABBIT', 'BALLENATO', 'TANDEM 2 flex', 'FOXY - Succionador con mango vibrador y orejitas']
descrip:
 [['No hay datos'], ['el superventas de amantis ¡mejorado!'], ['tu vibrador a distancia con aleta móvil y sumergible...'], ['vibrador doble flexible con mando'], ['No hay datos']]
precio:
 ['7.99', '39.99', '49.99', '49.99', '39.99']
URL:
 ['https://www.amantis.net/desliz-lubricante-intimo-agua-100ml/', 'https://www.amantis.net/tobogane-hot-rabbit-el-superventas-amantis-mejorado/', 'https://www.amantis.net/ballenato-tu-vibrador-distancia-aleta-movil-sumergible/']


#### c.- Extrayendo la información de los comentarios de cada producto.

Vamos a entrar en un producto para obtener la información de los comentarios que hay:
- Usuario
- Fecha
- Comentario
- Rating 

In [23]:
prueba=lista_URLs[1]
response = requests.get(prueba)
soup_prueba = bs(response.text, 'lxml')

titulo=soup_prueba.get_text(strip=True).split(',')[0]
print(titulo)

TOBOGANE HOT RABBIT


**Fechas**

Estos datos son en este momento, *string*.

In [24]:
rating=[]
all_ratings = soup_prueba.find_all("span", class_="date")  
sep_1=(' ')
sep_2=(', ')
# pattern = re.compile(sep)
for ratings in all_ratings:
    rating_coment_1=ratings.get_text(strip=True).split(sep_2)[1]
    rating_coment_2=ratings.get_text(strip=True).split(sep_2)[0]
    rating_coment_3=rating_coment_2.split(sep_1)[1:]
    date_1=rating_coment_3[0]+"/"+rating_coment_3[1]
    date_2=date_1+"/"+rating_coment_1                                       # Estamos pendientes de convertir a fechas, teniendo en cuenta
    # date_object = datetime.strptime(date_2,'%d%m%Y')                        # que esta en español
    # print(type(date_1))
    # print(date_1)
    rating.append(date_2)
    # print(type(date_2))
    # print(date_2)

print(len(rating))
rating

20


['22/noviembre/2022',
 '07/julio/2022',
 '23/abril/2022',
 '07/abril/2022',
 '26/marzo/2022',
 '31/enero/2022',
 '26/diciembre/2021',
 '20/diciembre/2021',
 '23/noviembre/2021',
 '08/octubre/2021',
 '13/septiembre/2021',
 '01/septiembre/2021',
 '11/agosto/2021',
 '07/junio/2021',
 '05/abril/2021',
 '05/marzo/2021',
 '26/noviembre/2020',
 '24/noviembre/2020',
 '12/noviembre/2020',
 '18/septiembre/2020']

**Usuarios**

In [25]:
user_comments=[]
all_user_comments = soup_prueba.find_all("span", class_="name-user")  

for user_comment in all_user_comments:
    user_comments.append(user_comment.get_text(strip=True))

print(len(user_comments))
user_comments

20


['Rossi',
 'Marina',
 'Jennifer',
 'Noa',
 'Karen',
 'Lorena',
 'JULIA',
 'Javier',
 'marta',
 'Michael',
 'David',
 'Marc',
 'Carla',
 'Anaïs',
 'Carla',
 'Roberto',
 'Gabriel',
 'Gabriel',
 'Marina',
 'Ana']

**Comentarios**

In [26]:
comment=[]
all_comments = soup_prueba.find_all("p")
for formats in all_comments[-len(rating):]:                     # Creo que me he cargado 'rating'
    comment.append(formats.get_text(strip=True))

print(comment[1])
len(comment)


Al principio me mosqueaban las orejitas, pero le he cogido el punto y ahora es mi juguete favorito. Ayufa usar lubricante para que no irrite por fuera, pero me parece genial. También el tamaño está bien. Otros más grandes, me parecen molestos... la opción calor me gusta, pero tarda un par de minutos y gasta mucha batería. Tampoco necesito las distintas vibraciones ya que me gusta el básico continuo. Aún así, un 10. rn


20

Hasta aquí los datos que estamos obteniendo se pasan a una lista por atributo.

### d.- Este es el código bueno para iterar todos los productos extraer la información siguiente:

- Nombre
- Descripción
- Precio sin rebaja (regular_price)
- Precio rebajado (new_price)
- Información de cada producto
- Lista de usuarios que han comentado
- Lista de comentarios
- Lista de Fecha de comentarios
- Lista de Ratings.


In [27]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
pages= np.arange(1,5)
# pages= np.arange(1, 25)

'''Listas a generar con la información de los productos'''
lista_URLs = []
name=[]
subname=[]
regular_prices=[]
new_price=[]
# lista_URLs = []
info=[]
charac=[]
user_comments=[]
comment=[]
date=[]
ratings=[]

''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)

'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]
    

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True).split(',')[0]
        name.append(nombre)
        sub_title=titulo.get_text(strip=True).split(', ')[1:]
        if sub_title==[]:
            sub_title=["No hay datos"]
        subname.append(sub_title)

    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:5]
    info.append(information)
    characteristic=description.get_text().split('\n')[5:-3]
    charac.append(characteristic)
    '''Vamos a obtener los datos de los comentarios de los usuarios'''

    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))
    user_comments.append(user_comments_product)

    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        # dates=datetime.strftime(dates, '%dd/%mm/%Y')
        date_comments_product.append(dates_text)
        # date_object = datetime.strptime(date_comments_product)
    date.append(date_comments_product)

    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))
    comment.append(comments_product)

    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = len(heart_rating)
        rating.append(num_hearts)
    ratings.append(rating)


for i, regular_price in enumerate(regular_prices):
    if regular_price is None:
        regular_prices[i] = new_price[i]


# print('-'*20)
# print(name)
print('-'*20)
print(len(name))
# print('-'*20)
# print(subname)
print('-'*20)
print(len(subname))
print('-'*20)
# print(regular_prices)
# print('-'*20)
print(len(regular_prices))
# print('-'*20)
# print(new_price)
print('-'*20)
print(len(new_price))
print('-'*20)
print(len(lista_URLs))
# print('-'*20)
# print(lista_URLs)
print('-'*20)
print(len(info))
# print('-'*20)
# # print(charac)
print('-'*20)
print(len(charac))
# print('-'*20)
# # print(user_comments)
print('-'*20)
print(len(user_comments))
# print('-'*20)
# # print(comment)
print('-'*20)
print(len(comment))
# # print('-'*20)
# # print(date)
print('-'*20)
print(len(date))
print('-'*20)
print(len(ratings))

Pagina 1
Pagina 2
Pagina 3
Pagina 4


KeyboardInterrupt: 

## Visualizamos los datos para ver qué se obtiene y lo guardamos en un fichero .csv.

La intención es depurar el codigo para un mejor manejo de los datos.

In [ ]:
# dataframe= pd.DataFrame({"Name": name,"Subname": subname,"Description": info, "Characteristics": charac,"Price":regular_price,"Reduced Price":new_price,
#                          "date":date,"User": user_comments,"Ratings": ratings,"Comment": comment})
# dataframe

,Name,Subname,Description,Characteristics,Price,Reduced Price,date,User,Ratings,Comment
0,TOBOGANE HOT RABBIT,[el superventas de amantis ¡mejorado!],[Vuelve nuestro vibrador de doble estimulación...,"[, , Medidas: 19cm (11cm insertables) y 3,3cm/...",14.99,39.99,"[martes 22 noviembre, 2022, jueves 07 julio, 2...","[Rossi, Marina, Jennifer, Noa, Karen, Lorena, ...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","[Mi primera compra. Me encantó la textura, los..."
1,BALLENATO,[tu vibrador a distancia con aleta móvil y sum...,[De las profundidades más húmedas llega BALLEN...,[¡FREE BALLENATO! Ver características y medida...,14.99,49.99,"[sábado 18 febrero, 2023, miércoles 01 febrero...","[Almudena, Tomabel, andrea, Carlos, maria, Mar...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, ...","[Sigo temblando con este juguete, menudas vibr..."
2,TOBOGANE,[el vibrador doble más vendido],"[Por favor, desabróchense los cinturones de se...","[, La última atracción exclusiva de amantis es...",14.99,36.99,"[lunes 06 marzo, 2023, sábado 04 marzo, 2023, ...","[Teresa, Alicia, María, Sara, Raquel, Sara, Da...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, ...",[Fue el primer vibrador que compré. Me lo reco...
3,MENEO sube y baja,[placer realista con control remoto],"[Si te gusta que te metan un buen meneo, hazte...","[Ya lo ves, Meneo’s Cock puede ir contigo a do...",14.99,44.99,"[miércoles 14 diciembre, 2022, miércoles 09 no...","[Francisco, Maria, Jose Javier, Carlos, Pedro,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",[Quería saber cuantos cm tiene la longitud que...
4,FRESH GIRL,[6 Kilos y 40cm de piel real disfrutable],[FRESH GIRL es una bocanada de aire fresco en ...,[FRESH GIRL es tan pesada que tendrás que usar...,14.99,99.99,"[sábado 31 diciembre, 2022, viernes 21 octubre...","[Adrian, Daniel, victor, Guillermo, Jesús, Dav...","[4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[Este juguete es bastante bueno, muy muy place..."
...,...,...,...,...,...,...,...,...,...,...
91,ARMOUR ALAS - Armadura sexual BDSM con alas,[No hay datos],[Las prendas de la cultura BDSM han inundado l...,[Este modelo no es una única pieza si no que e...,14.99,29.99,[],[],[],[Parece que tu navegador está bloqueando JavaS...
92,LAZZO,[Atar tu pene para desatar tu pasión],[Te agarra la base del pene pero para dar rien...,"[Ver características y medidas, , \r, Funda pa...",14.99,12.99,"[sábado 30 abril, 2022, martes 23 noviembre, 2...","[Javier, Miguel]","[4, 4]","[Está hecho de un buen material, es muy elásti..."
93,MYSTERY,[body de rejilla super sexy],[El poder de la seducción está en tus manos co...,"[Siéntente más sexy que nunca, sin renunciar a...",14.99,4.99,"[viernes 09 diciembre, 2022, lunes 28 marzo, 2...","[andrea, María, María, Javier, Lucia, Anonimo]","[4, 3, 5, 4, 5, 5]","[es totalmente precioso, queda estupendo y a m..."
94,Arnés sexual TRIANGLE,[100% Vegano con brillo y sujeción firme],[En los juegos del amarse y disfrutarse todo p...,[Puedes usar este arnés con dildos de base anc...,14.99,12.99,"[sábado 10 diciembre, 2022, jueves 06 octubre,...","[Sara, Sergio, Silvia, Eme, Ramón, Alba, Anne,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]",[Es la segunda vez que lo compro y estoy encan...


In [ ]:
# file=folder+'\\Data\\scrapped_data.csv'
# file

In [ ]:
# dataframe.to_csv(file,header=True,index=False)           # Tengo que generar el path correcto


## Solucionando inconvenientes de los datos obtenidos.

- *Subname* puede generar problemas por lo eliminaremos del codigo. Lo generaremos después a partir de *Name*.
- *Characteristics* puede generar problemas, ya que serían datos de dimensiones, duración, etc. Lo generaremos después a partir de *Description*.
- Hay que pasar las listas a registros individuales.

In [ ]:
lista_URLs[0]

In [ ]:
# url = "https://www.amantis.net/productos-amantis/"              # lista productos
# url_principal="https://www.amantis.net/"                        # productos
# # pages= np.arange(1,5)
# pages= np.arange(1, 25)

# '''Listas a generar con la información de los productos'''
# lista_URLs = []

# ''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

# for page in pages:
#     if page == 1:
#         print("Leyendo paginas")
#         # print("Pagina",page)
#         URL = url
#         response = requests.get(url)
#         soup = bs(response.text, 'lxml')
#         productos = soup.find_all(class_='caption')
#         for producto in productos[9:]:
#             URL_producto = producto.find('a')['href']
#             lista_URLs.append(URL_producto)
        
#     else:
# #        print("Pagina",page)
#         URL = url+'page' + str(page)+'/'
#         response = requests.get(URL)
#         soup = bs(response.text, 'lxml')
#         productos = soup.find_all(class_='caption')
#         for producto in productos[9:]:
#             URL_producto = producto.find('a')['href']
#             lista_URLs.append(URL_producto)
# print("Terminando lectura.\nRecabando información.")

# list_name=[]
# list_regular_prices=[]
# list_new_price=[]
# list_info=[]
# user_comments=[]
# comment=[]
# date=[]
# ratings=[]
# datas=[]
# diccionario_URL={}



# '''Extraemos la información de cada producto existente'''

# for URL in lista_URLs:
#     url_product=URL
#     response_product = requests.get(url_product)
#     soup_product = bs(response_product.text, 'lxml')
#     user_comments_product=[]
#     date_comments_product=[]
#     comments_product=[]
#     rating=[]
#     data=[]
#     name=[]
#     regular_prices=[]
#     new_price=[]
#     info=[]

#     diccionario_comments={}

#     titulos=soup_product.find_all("h1",class_="h3")
#     for titulo in titulos:
#         nombre=titulo.get_text(strip=True)
#         name.append(nombre)

#     description=soup_product.find("div", class_="description") 
#     information=description.get_text().split('\n')[1:]
#     documentation = ''.join(information)
#     info.append(documentation)


#     all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
#     for price_container in all_price:                                                                    
#         try:
#             special_price = price_container.find("span", class_="productSpecialPrice")
#             if special_price:
#                 item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
#                 new_price.append(item_price)
#                 regular_price = price_container.find("del").get_text(strip=True)
#                 item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
#                 regular_prices.append(item_regular_price)
#             else:
#                 regular_price = price_container.find("span").get_text(strip=True)
#                 item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
#                 new_price.append(item_regular_price)
#                 regular_prices.append(None)
#         except:
#             new_price.append(None)
#             regular_prices.append(None)

#     for i, regular_price in enumerate(regular_prices):
#         if regular_price is None:
#             regular_prices[i] = new_price[i]


#     '''Vamos a obtener los datos de los comentarios de los usuarios'''
#     all_user_comments = soup_product.find_all("span", class_="name-user") 
#     for user_comment in all_user_comments:
#         user_comments_product.append(user_comment.get_text(strip=True))
#     all_dates = soup_product.find_all("span", class_="date")  
#     for dates in all_dates:
#         dates_text=dates.get_text(strip=True)
#         date_comments_product.append(dates_text)
#     all_comments = soup_product.find_all("p")
#     for formats in all_comments[-len(date_comments_product):]:
#         comments_product.append(formats.get_text(strip=True))
#     hearts = soup_product.find_all('div', class_= 'box-description')
#     for heart in hearts:
#         heart_rating = heart.find_all('span', class_= 'fas fa-heart')
#         num_hearts = str(len(heart_rating))
#         rating.append(num_hearts)
    
#     # diccionario={}
#     user_comments.append(user_comments_product)
#     date.append(date_comments_product)
#     comment.append(comments_product)
#     ratings.append(rating)
    
#     # datos = list(zip(nombre,user_comments_product,comments_product, date_comments_product, rating))
#     # for dato in datos:
#     #     nombre=dato[0]
#     #     comentarios=dato[1:]
#     #     if nombre in diccionario_comments:
#     #         diccionario_comments[nombre].append(comentarios)
#     #     else:
#     #         diccionario_comments[nombre] = [comentarios]
#     # # diccionario_URL[name] = name
#     # diccionario_URL[URL] = diccionario_comments
    
    
# # diccionario_URL


# dataframe= pd.DataFrame({"Name": name,"Description": info,"Price":regular_price,"Reduced Price":new_price,
#                          "date":date,"User": user_comments,"Ratings": ratings,"Comment": comment
#                         #  ,"diccionario":diccionario
#                         })
# dataframe

### Generando 2 diccionarios para separar la información y analizar la información dentro de estos diccionarios.

Vamos a generar a partir de estos dos diccionarios, 2 dataframes para su estudio, pasarlo a un fichero .csv para guardarlo en una BBDD para su tratamiento posterior.

In [3]:
url = "https://www.amantis.net/productos-amantis/"              # lista productos
url_principal="https://www.amantis.net/"                        # productos
# pages= np.arange(1,5)
pages= np.arange(1, 25)

'''Listas a generar con la información de los productos'''
lista_URLs = []
name=[]
regular_prices=[]
new_price=[]
info=[]
user_comments=[]
comment=[]
date=[]
ratings=[]
id=[]
comentarios=[]



'''Generamos 2 diccionarios con los datos importantes para ingresar en una BBDD'''

diccionario_datos_productos={"ID":id,"NAME":name,"INFO":info,"LISTA_URL":lista_URLs,"REGULAR_PRICE":regular_prices,"DISCOUNT_PRICE":new_price}

diccionario_comentarios_productos={"ID":id,"COMENTARIOS":comentarios}



''' Obtenemos las URLs de los productos para entrar luego en sus URLS y extraer la información'''

for page in pages:
    if page == 1:
        print("Pagina",page)
        URL = url
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)
        
    else:
        print("Pagina",page)
        URL = url+'page' + str(page)+'/'
        response = requests.get(URL)
        soup = bs(response.text, 'lxml')
        productos = soup.find_all(class_='caption')
        for producto in productos[9:]:
            URL_producto = producto.find('a')['href']
            lista_URLs.append(URL_producto)


for i in range(len(lista_URLs)):
    id.append(i)

    
'''Extraemos la información de cada producto existente'''

for URL in lista_URLs:
    url_product=URL
    response_product = requests.get(url_product)
    soup_product = bs(response_product.text, 'lxml')
    user_comments_product=[]
    date_comments_product=[]
    comments_product=[]
    rating=[]

    titulos=soup_product.find_all("h1",class_="h3")
    for titulo in titulos:
        nombre=titulo.get_text(strip=True)
        name.append(nombre)

    all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
    for price_container in all_price:                                                                    
        try:
            special_price = price_container.find("span", class_="productSpecialPrice")
            if special_price:
                item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                new_price.append(item_price)
                regular_price = price_container.find("del").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                regular_prices.append(item_regular_price)
            else:
                regular_price = price_container.find("span").get_text(strip=True)
                item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                new_price.append(item_regular_price)
                regular_prices.append(None)
        except:
            new_price.append(None)
            regular_prices.append(None)

    description=soup_product.find("div", class_="description") 
    information=description.get_text().split('\n')[1:]
    documentation = ''.join(information)
    info.append(documentation)


    '''Vamos a obtener los datos de los comentarios de los usuarios'''

    all_user_comments = soup_product.find_all("span", class_="name-user") 
    for user_comment in all_user_comments:
        user_comments_product.append(user_comment.get_text(strip=True))
    user_comments.append(user_comments_product)

    all_dates = soup_product.find_all("span", class_="date")  
    for dates in all_dates:
        dates_text=dates.get_text(strip=True)
        # dates=datetime.strftime(dates, '%dd/%mm/%Y')
        date_comments_product.append(dates_text)
        # date_object = datetime.strptime(date_comments_product)
    date.append(date_comments_product)

    all_comments = soup_product.find_all("p")
    for formats in all_comments[-len(date_comments_product):]:
        comments_product.append(formats.get_text(strip=True))
    comment.append(comments_product)

    hearts = soup_product.find_all('div', class_= 'box-description')
    for heart in hearts:
        heart_rating = heart.find_all('span', class_= 'fas fa-heart')
        num_hearts = len(heart_rating)
        rating.append(num_hearts)
    ratings.append(rating)

    datos = list(zip( date_comments_product,rating, user_comments_product,comments_product ))
    comentarios.append(datos)

for i, regular_price in enumerate(regular_prices):
    if regular_price is None:
        regular_prices[i] = new_price[i]


print('-'*20)
print(len(name))
print('-'*20)
print(len(regular_prices))
print('-'*20)
print(len(new_price))
print('-'*20)
print(len(lista_URLs))
print('-'*20)
print(len(info))
print('-'*20)
print(len(user_comments))
print('-'*20)
print(len(comment))
print('-'*20)
print(len(date))
print('-'*20)
print(len(ratings))

Pagina 1
Pagina 2
Pagina 3
Pagina 4
Pagina 5
Pagina 6
Pagina 7
Pagina 8
Pagina 9
Pagina 10
Pagina 11
Pagina 12
Pagina 13
Pagina 14
Pagina 15
Pagina 16
Pagina 17
Pagina 18
Pagina 19
Pagina 20
Pagina 21
Pagina 22
Pagina 23
Pagina 24
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576
--------------------
576


In [4]:
productos=pd.DataFrame(diccionario_datos_productos)
productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              576 non-null    int64  
 1   NAME            576 non-null    object 
 2   INFO            576 non-null    object 
 3   LISTA_URL       576 non-null    object 
 4   REGULAR_PRICE   576 non-null    float64
 5   DISCOUNT_PRICE  576 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 27.1+ KB


### Buscando duplicados para eliminarlos

In [5]:
productos.NAME.value_counts()


OISHi, Camiseta de rejilla negra con mangas                     2
WARRIOR, suave casco vibrador para masaje integral              2
Columpio sexual de amantis, el asistente postural definitivo    2
GALGA, sofisticadas braguitas-arnés de amantis                  2
DOBERMANA, arnés fino y de doble anilla                         2
                                                               ..
SAREA, body de calentamiento de rejilla ancha                   1
JUSTISSE CHOCKER, unisex con argolla                            1
ADA, Vagina translúcida extra suave de amantis                  1
LOCOMOTION rugged, tu sex-machine con vibración y ventosa       1
JUSTISSE BACK - mordaza de bola con esposas                     1
Name: NAME, Length: 550, dtype: int64

In [6]:
noduplicated_product = productos.drop_duplicates(subset='NAME', keep='first')
noduplicated_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 575
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              550 non-null    int64  
 1   NAME            550 non-null    object 
 2   INFO            550 non-null    object 
 3   LISTA_URL       550 non-null    object 
 4   REGULAR_PRICE   550 non-null    float64
 5   DISCOUNT_PRICE  550 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 30.1+ KB


Determinamos los ID de los registros a eliminar

In [7]:
removed_id = productos[productos.duplicated(subset='NAME', keep='first')]['ID']
removed_id.info()

<class 'pandas.core.series.Series'>
Int64Index: 26 entries, 26 to 224
Series name: ID
Non-Null Count  Dtype
--------------  -----
26 non-null     int64
dtypes: int64(1)
memory usage: 416.0 bytes


In [8]:
removed_id

26      26
27      27
48      48
49      49
73      73
75      75
79      79
82      82
98      98
99      99
122    122
124    124
132    132
141    141
155    155
168    168
170    170
172    172
173    173
176    176
178    178
181    181
195    195
196    196
221    221
224    224
Name: ID, dtype: int64

Aqui genero un dataFrame con los datos de los comentarios.

En este dataframe creado no están los id de los productos que no tienen comentarios.

In [9]:
comentarios_productos=pd.DataFrame(diccionario_comentarios_productos)

id=[]
comments=[]
date=[]
ratio=[]
users=[]
comment=[]

comentarios=pd.DataFrame()
diccionario={"id":id,"comments":comments}

for id_product,n_comments in enumerate (comentarios_productos['COMENTARIOS']):
    # print("Imprimiendo texto del indice",id_product)
    # print("Imprimiento n_comentarios",len(n_comments))
    for i in n_comments:
        # print("id",id_product,"coments",comments)
        id.append(id_product)
        comments.append(i)


for j in range(len(diccionario['comments'])):
    date.append(diccionario['comments'][j][0])
    ratio.append(diccionario['comments'][j][1])
    users.append(diccionario['comments'][j][2])
    comment.append(diccionario['comments'][j][3])


comentarios['ID']=pd.Series(id)
comentarios['DATE']=pd.Series(date)
comentarios['RATIO']=pd.Series(ratio)
comentarios['USERS']=pd.Series(users)
comentarios['COMMENT']=pd.Series(comment)
comentarios.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11551 entries, 0 to 11550
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       11551 non-null  int64 
 1   DATE     11551 non-null  object
 2   RATIO    11551 non-null  int64 
 3   USERS    11551 non-null  object
 4   COMMENT  11551 non-null  object
dtypes: int64(2), object(3)
memory usage: 451.3+ KB


In [10]:
comentarios.describe()

,ID,RATIO
count,11551.000000,11551.000000
mean,206.809540,4.669812
std,144.654198,0.683426
min,0.000000,1.000000
25%,114.000000,5.000000
50%,182.000000,5.000000
75%,320.000000,5.000000
max,575.000000,5.000000


In [11]:
comentarios.tail()
# productos.tail()

,ID,DATE,RATIO,USERS,COMMENT
11546,574,"jueves 27 junio, 2013",5,Laura,"Una monada, quedan super sexy y genial para ll..."
11547,574,"viernes 01 febrero, 2013",5,miguel,Hola este producto lo compramos y ese mismo dí...
11548,575,"jueves 22 diciembre, 2022",3,Marta,Lo malo de este juguete es la bola de la morda...
11549,575,"domingo 18 febrero, 2018",4,Héctor,Te limita un poco el movimiento... pero de eso...
11550,575,"lunes 14 noviembre, 2016",5,Francisco,"me encanta. En la foto no se aprecia, pero se ..."


### Elimino los registros de ID que ya he eliminado en el anterior dataframe.

In [12]:
noduplicated_comments = comentarios[~comentarios['ID'].isin(productos[productos['ID'].isin(removed_id)]['ID'])]
noduplicated_comments.info()
# Imprimir los resultados

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10659 entries, 0 to 11550
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       10659 non-null  int64 
 1   DATE     10659 non-null  object
 2   RATIO    10659 non-null  int64 
 3   USERS    10659 non-null  object
 4   COMMENT  10659 non-null  object
dtypes: int64(2), object(3)
memory usage: 499.6+ KB


In [13]:
noduplicated_product.to_csv('./Data/productos_scrape.csv',header=True,index=False)           # Tengo que generar el path correcto
noduplicated_comments.to_csv('./Data/comentarios_scrape.csv',header=True,index=False)           # Tengo que generar el path correcto